# 왜 학습용 데이터랑 테스트 데이터랑 안 나눴니... 나 자신아!

In [1]:
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import subprocess
import os

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
np.random.seed(1)

In [3]:
np.random.rand(5)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
       1.46755891e-01])

In [7]:
log_files = os.listdir('../logSplitDataset')
len(log_files)

3730

In [6]:
test_log_files = random.sample(log_files, int(len(log_files) * 0.2))
len(test_log_files)

746

In [8]:
print(len(log_files))
print(len(test_log_files))

3730
746


In [9]:
train_log_files = log_files
for i in test_log_files:
    train_log_files.remove(i)
    
print(len(train_log_files))

2984


In [10]:
test_log_files.sort()
test_log_files

['.DS_Store',
 '2020-08-24_10-02-1.csv',
 '2020-08-24_10-03-1.csv',
 '2020-08-24_10-16-1.csv',
 '2020-08-24_10-34-1.csv',
 '2020-08-24_10-35-1.csv',
 '2020-08-24_10-41-1.csv',
 '2020-08-24_10-42-1.csv',
 '2020-08-24_10-43-1.csv',
 '2020-08-24_10-44-1.csv',
 '2020-08-24_10-59-1.csv',
 '2020-08-24_11-07-1.csv',
 '2020-08-24_11-09-1.csv',
 '2020-08-24_11-10-1.csv',
 '2020-08-24_11-15-1.csv',
 '2020-08-24_11-27-1.csv',
 '2020-08-24_11-29-1.csv',
 '2020-08-24_11-37-1.csv',
 '2020-08-24_11-49-1.csv',
 '2020-08-24_11-54-1.csv',
 '2020-08-24_11-57-0.csv',
 '2020-08-24_12-01-0.csv',
 '2020-08-24_12-02-1.csv',
 '2020-08-24_12-06-1.csv',
 '2020-08-24_12-20-1.csv',
 '2020-08-24_12-22-1.csv',
 '2020-08-24_12-32-1.csv',
 '2020-08-24_12-42-1.csv',
 '2020-08-24_12-45-0.csv',
 '2020-08-24_12-50-1.csv',
 '2020-08-24_12-52-1.csv',
 '2020-08-24_12-53-1.csv',
 '2020-08-24_12-54-1.csv',
 '2020-08-24_13-07-1.csv',
 '2020-08-24_13-29-1.csv',
 '2020-08-24_13-32-1.csv',
 '2020-08-24_13-34-1.csv',
 '2020-08-24_1

In [11]:
train_log_files.sort()
train_log_files[4][-5]

'1'

# 분리한 파일명으로 각각 다른 폴더에 저장하기

In [17]:
n = 0
m = 0
for fileName in test_log_files:
    imgFileName = fileName.replace('.csv', '.png')
    
    out = subprocess.check_output(['xxd','-p',f'../logSplitDataset/{fileName}'])
    out = str(out)
#     print(out)
    out = out[2:-1].replace('\\n', '')
    length = 2
    outListHex = list(map(''.join, zip(*[iter(out)]*length)))
    outListInt = [int(x, 16) for x in outListHex]
    
    if len(outListInt) % 4 == 1:
        for i in range(3):
            outListInt.append(0)

    elif len(outListInt) % 4 == 2:
        for i in range(2):
            outListInt.append(0)

    elif len(outListInt) % 4 == 3:
        for i in range(1):
            outListInt.append(0)
            
    outArray = np.array(outListInt)
    outArray = outArray.reshape(-1,4)
    
    empty = np.zeros([128, 128])
    empty = empty.astype('uint8')
    for x1, y1, x2, y2 in outArray:
        empty[x1:y1, x2:y2] += 1

    if imgFileName[-5] == '1':
        cv2.imwrite('../img_dataset/test/attack/attack_{0:04d}.png'.format(n), empty)
        n += 1
    else:
        cv2.imwrite('../img_dataset/test/normal/normal_{0:04d}.png'.format(m), empty)
        m += 1

In [20]:
n = 0
m = 0
for fileName in train_log_files:
    imgFileName = fileName.replace('.csv', '.png')
    
    out = subprocess.check_output(['xxd','-p',f'../logSplitDataset/{fileName}'])
    out = str(out)
#     print(out)
    out = out[2:-1].replace('\\n', '')
    length = 2
    outListHex = list(map(''.join, zip(*[iter(out)]*length)))
    outListInt = [int(x, 16) for x in outListHex]
    
    if len(outListInt) % 4 == 1:
        for i in range(3):
            outListInt.append(0)

    elif len(outListInt) % 4 == 2:
        for i in range(2):
            outListInt.append(0)

    elif len(outListInt) % 4 == 3:
        for i in range(1):
            outListInt.append(0)
        
    outArray = np.array(outListInt)
    outArray = outArray.reshape(-1,4)
    
    empty = np.zeros([128, 128])
    empty = empty.astype('uint8')
    for x1, y1, x2, y2 in outArray:
        empty[x1:y1, x2:y2] += 1
#         cv2.circle(empty, (x, y), 8, (255,255,255), 1)
#     print(empty)
    if imgFileName[-5] == '1':
        cv2.imwrite('../img_dataset/train/attack/attack_{0:04d}.png'.format(n), empty)
        n += 1
    else:
        cv2.imwrite('../img_dataset/train/normal/normal_{0:04d}.png'.format(m), empty)
        m += 1

# 이제 나눠진 파일로 다시 모델을 만들어보자!


In [2]:
import os

In [3]:
print(len(os.listdir('../imgDataset/test/attack')))
print(len(os.listdir('../imgDataset/test/normal')))
print(len(os.listdir('../imgDataset/train/attack')))
print(len(os.listdir('../imgDataset/train/normal')))

299
494
1225
1948


In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [5]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 이전 CNN 레이어에서 나온 3차원 배열은 1차원으로 뽑아줍니다
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
batch_size = 16

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다.
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        '../imgDataset/train',  # this is the target directory
        target_size=(128, 128),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        '../imgDataset/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        '../imgDataset/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

Found 3172 images belonging to 2 classes.
Found 793 images belonging to 2 classes.
Found 793 images belonging to 2 classes.


In [15]:
model.fit(train_generator, steps_per_epoch=batch_size, validation_data=validation_generator, epochs=1)
model.save_weights('conan_ver_02.h5')

ValueError: in user code:

    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_

In [ ]:
# X는 이미지 행렬 y는 라벨링

In [37]:
import os
 
def changeName(path, cName):
    i = 0
    for filename in os.listdir(path):
        print(path+filename, '=>', path+str(cName)+'_{0:04d}'.format(i)+'.png')
        os.rename(path+filename, path+str(cName)+'_{0:04d}'.format(i)+'.png')
        i += 1
        

In [38]:
changeName('../imgDataset/test/attack/','attack')

../imgDataset/test/attack/attack_230.png => ../imgDataset/test/attack/attack_0000.png
../imgDataset/test/attack/attack_224.png => ../imgDataset/test/attack/attack_0001.png
../imgDataset/test/attack/attack_218.png => ../imgDataset/test/attack/attack_0002.png
../imgDataset/test/attack/attack_146.png => ../imgDataset/test/attack/attack_0003.png
../imgDataset/test/attack/attack_29.png => ../imgDataset/test/attack/attack_0004.png
../imgDataset/test/attack/attack_152.png => ../imgDataset/test/attack/attack_0005.png
../imgDataset/test/attack/attack_15.png => ../imgDataset/test/attack/attack_0006.png
../imgDataset/test/attack/attack_185.png => ../imgDataset/test/attack/attack_0007.png
../imgDataset/test/attack/attack_191.png => ../imgDataset/test/attack/attack_0008.png
../imgDataset/test/attack/attack_190.png => ../imgDataset/test/attack/attack_0009.png
../imgDataset/test/attack/attack_184.png => ../imgDataset/test/attack/attack_0010.png
../imgDataset/test/attack/attack_14.png => ../imgDataset

In [39]:
changeName('../imgDataset/test/normal/','normal')

../imgDataset/test/normal/normal_82.png => ../imgDataset/test/normal/normal_0000.png
../imgDataset/test/normal/normal_96.png => ../imgDataset/test/normal/normal_0001.png
../imgDataset/test/normal/normal_41.png => ../imgDataset/test/normal/normal_0002.png
../imgDataset/test/normal/normal_128.png => ../imgDataset/test/normal/normal_0003.png
../imgDataset/test/normal/normal_55.png => ../imgDataset/test/normal/normal_0004.png
../imgDataset/test/normal/normal_114.png => ../imgDataset/test/normal/normal_0005.png
../imgDataset/test/normal/normal_69.png => ../imgDataset/test/normal/normal_0006.png
../imgDataset/test/normal/normal_100.png => ../imgDataset/test/normal/normal_0007.png
../imgDataset/test/normal/normal_458.png => ../imgDataset/test/normal/normal_0008.png
../imgDataset/test/normal/normal_316.png => ../imgDataset/test/normal/normal_0009.png
../imgDataset/test/normal/normal_470.png => ../imgDataset/test/normal/normal_0010.png
../imgDataset/test/normal/normal_464.png => ../imgDataset/t

../imgDataset/test/normal/normal_392.png => ../imgDataset/test/normal/normal_0335.png
../imgDataset/test/normal/normal_386.png => ../imgDataset/test/normal/normal_0336.png
../imgDataset/test/normal/normal_351.png => ../imgDataset/test/normal/normal_0337.png
../imgDataset/test/normal/normal_437.png => ../imgDataset/test/normal/normal_0338.png
../imgDataset/test/normal/normal_423.png => ../imgDataset/test/normal/normal_0339.png
../imgDataset/test/normal/normal_345.png => ../imgDataset/test/normal/normal_0340.png
../imgDataset/test/normal/normal_379.png => ../imgDataset/test/normal/normal_0341.png
../imgDataset/test/normal/normal_225.png => ../imgDataset/test/normal/normal_0342.png
../imgDataset/test/normal/normal_231.png => ../imgDataset/test/normal/normal_0343.png
../imgDataset/test/normal/normal_219.png => ../imgDataset/test/normal/normal_0344.png
../imgDataset/test/normal/normal_7.png => ../imgDataset/test/normal/normal_0345.png
../imgDataset/test/normal/normal_6.png => ../imgDataset/

In [40]:
changeName('../imgDataset/train/normal/','normal')

../imgDataset/train/normal/normal_855.png => ../imgDataset/train/normal/normal_0000.png
../imgDataset/train/normal/normal_82.png => ../imgDataset/train/normal/normal_0001.png
../imgDataset/train/normal/normal_96.png => ../imgDataset/train/normal/normal_0002.png
../imgDataset/train/normal/normal_699.png => ../imgDataset/train/normal/normal_0003.png
../imgDataset/train/normal/normal_841.png => ../imgDataset/train/normal/normal_0004.png
../imgDataset/train/normal/normal_869.png => ../imgDataset/train/normal/normal_0005.png
../imgDataset/train/normal/normal_1486.png => ../imgDataset/train/normal/normal_0006.png
../imgDataset/train/normal/normal_1492.png => ../imgDataset/train/normal/normal_0007.png
../imgDataset/train/normal/normal_896.png => ../imgDataset/train/normal/normal_0008.png
../imgDataset/train/normal/normal_41.png => ../imgDataset/train/normal/normal_0009.png
../imgDataset/train/normal/normal_1479.png => ../imgDataset/train/normal/normal_0010.png
../imgDataset/train/normal/norma

../imgDataset/train/normal/normal_271.png => ../imgDataset/train/normal/normal_0339.png
../imgDataset/train/normal/normal_1913.png => ../imgDataset/train/normal/normal_0340.png
../imgDataset/train/normal/normal_1907.png => ../imgDataset/train/normal/normal_0341.png
../imgDataset/train/normal/normal_1091.png => ../imgDataset/train/normal/normal_0342.png
../imgDataset/train/normal/normal_1085.png => ../imgDataset/train/normal/normal_0343.png
../imgDataset/train/normal/normal_339.png => ../imgDataset/train/normal/normal_0344.png
../imgDataset/train/normal/normal_1668.png => ../imgDataset/train/normal/normal_0345.png
../imgDataset/train/normal/normal_311.png => ../imgDataset/train/normal/normal_0346.png
../imgDataset/train/normal/normal_1640.png => ../imgDataset/train/normal/normal_0347.png
../imgDataset/train/normal/normal_1898.png => ../imgDataset/train/normal/normal_0348.png
../imgDataset/train/normal/normal_1126.png => ../imgDataset/train/normal/normal_0349.png
../imgDataset/train/norm

../imgDataset/train/normal/normal_92.png => ../imgDataset/train/normal/normal_0505.png
../imgDataset/train/normal/normal_86.png => ../imgDataset/train/normal/normal_0506.png
../imgDataset/train/normal/normal_851.png => ../imgDataset/train/normal/normal_0507.png
../imgDataset/train/normal/normal_689.png => ../imgDataset/train/normal/normal_0508.png
../imgDataset/train/normal/normal_879.png => ../imgDataset/train/normal/normal_0509.png
../imgDataset/train/normal/normal_1496.png => ../imgDataset/train/normal/normal_0510.png
../imgDataset/train/normal/normal_1482.png => ../imgDataset/train/normal/normal_0511.png
../imgDataset/train/normal/normal_188.png => ../imgDataset/train/normal/normal_0512.png
../imgDataset/train/normal/normal_836.png => ../imgDataset/train/normal/normal_0513.png
../imgDataset/train/normal/normal_822.png => ../imgDataset/train/normal/normal_0514.png
../imgDataset/train/normal/normal_1383.png => ../imgDataset/train/normal/normal_0515.png
../imgDataset/train/normal/norm

../imgDataset/train/normal/normal_1035.png => ../imgDataset/train/normal/normal_0672.png
../imgDataset/train/normal/normal_955.png => ../imgDataset/train/normal/normal_0673.png
../imgDataset/train/normal/normal_941.png => ../imgDataset/train/normal/normal_0674.png
../imgDataset/train/normal/normal_799.png => ../imgDataset/train/normal/normal_0675.png
../imgDataset/train/normal/normal_1586.png => ../imgDataset/train/normal/normal_0676.png
../imgDataset/train/normal/normal_969.png => ../imgDataset/train/normal/normal_0677.png
../imgDataset/train/normal/normal_1592.png => ../imgDataset/train/normal/normal_0678.png
../imgDataset/train/normal/normal_8.png => ../imgDataset/train/normal/normal_0679.png
../imgDataset/train/normal/normal_1579.png => ../imgDataset/train/normal/normal_0680.png
../imgDataset/train/normal/normal_996.png => ../imgDataset/train/normal/normal_0681.png
../imgDataset/train/normal/normal_982.png => ../imgDataset/train/normal/normal_0682.png
../imgDataset/train/normal/nor

../imgDataset/train/normal/normal_1192.png => ../imgDataset/train/normal/normal_0853.png
../imgDataset/train/normal/normal_25.png => ../imgDataset/train/normal/normal_0854.png
../imgDataset/train/normal/normal_1409.png => ../imgDataset/train/normal/normal_0855.png
../imgDataset/train/normal/normal_158.png => ../imgDataset/train/normal/normal_0856.png
../imgDataset/train/normal/normal_31.png => ../imgDataset/train/normal/normal_0857.png
../imgDataset/train/normal/normal_1421.png => ../imgDataset/train/normal/normal_0858.png
../imgDataset/train/normal/normal_170.png => ../imgDataset/train/normal/normal_0859.png
../imgDataset/train/normal/normal_19.png => ../imgDataset/train/normal/normal_0860.png
../imgDataset/train/normal/normal_616.png => ../imgDataset/train/normal/normal_0861.png
../imgDataset/train/normal/normal_1347.png => ../imgDataset/train/normal/normal_0862.png
../imgDataset/train/normal/normal_602.png => ../imgDataset/train/normal/normal_0863.png
../imgDataset/train/normal/norm

...

../imgDataset/train/normal/normal_597.png => ../imgDataset/train/normal/normal_1257.png
../imgDataset/train/normal/normal_583.png => ../imgDataset/train/normal/normal_1258.png
../imgDataset/train/normal/normal_1944.png => ../imgDataset/train/normal/normal_1259.png
../imgDataset/train/normal/normal_1788.png => ../imgDataset/train/normal/normal_1260.png
../imgDataset/train/normal/normal_232.png => ../imgDataset/train/normal/normal_1261.png
../imgDataset/train/normal/normal_1763.png => ../imgDataset/train/normal/normal_1262.png
../imgDataset/train/normal/normal_1005.png => ../imgDataset/train/normal/normal_1263.png
../imgDataset/train/normal/normal_554.png => ../imgDataset/train/normal/normal_1264.png
../imgDataset/train/normal/normal_1011.png => ../imgDataset/train/normal/normal_1265.png
../imgDataset/train/normal/normal_540.png => ../imgDataset/train/normal/normal_1266.png
../imgDataset/train/normal/normal_226.png => ../imgDataset/train/normal/normal_1267.png
../imgDataset/train/normal/

../imgDataset/train/normal/normal_218.png => ../imgDataset/train/normal/normal_1418.png
../imgDataset/train/normal/normal_556.png => ../imgDataset/train/normal/normal_1419.png
../imgDataset/train/normal/normal_1007.png => ../imgDataset/train/normal/normal_1420.png
../imgDataset/train/normal/normal_1761.png => ../imgDataset/train/normal/normal_1421.png
../imgDataset/train/normal/normal_230.png => ../imgDataset/train/normal/normal_1422.png
../imgDataset/train/normal/normal_1775.png => ../imgDataset/train/normal/normal_1423.png
../imgDataset/train/normal/normal_224.png => ../imgDataset/train/normal/normal_1424.png
../imgDataset/train/normal/normal_542.png => ../imgDataset/train/normal/normal_1425.png
../imgDataset/train/normal/normal_1013.png => ../imgDataset/train/normal/normal_1426.png
../imgDataset/train/normal/normal_1946.png => ../imgDataset/train/normal/normal_1427.png
../imgDataset/train/normal/normal_595.png => ../imgDataset/train/normal/normal_1428.png
../imgDataset/train/normal/

../imgDataset/train/normal/normal_283.png => ../imgDataset/train/normal/normal_1623.png
../imgDataset/train/normal/normal_1739.png => ../imgDataset/train/normal/normal_1624.png
../imgDataset/train/normal/normal_268.png => ../imgDataset/train/normal/normal_1625.png
../imgDataset/train/normal/normal_1705.png => ../imgDataset/train/normal/normal_1626.png
../imgDataset/train/normal/normal_254.png => ../imgDataset/train/normal/normal_1627.png
../imgDataset/train/normal/normal_532.png => ../imgDataset/train/normal/normal_1628.png
../imgDataset/train/normal/normal_1063.png => ../imgDataset/train/normal/normal_1629.png
../imgDataset/train/normal/normal_526.png => ../imgDataset/train/normal/normal_1630.png
../imgDataset/train/normal/normal_1077.png => ../imgDataset/train/normal/normal_1631.png
../imgDataset/train/normal/normal_1711.png => ../imgDataset/train/normal/normal_1632.png
../imgDataset/train/normal/normal_240.png => ../imgDataset/train/normal/normal_1633.png
../imgDataset/train/normal/

../imgDataset/train/normal/normal_1113.png => ../imgDataset/train/normal/normal_1797.png
../imgDataset/train/normal/normal_442.png => ../imgDataset/train/normal/normal_1798.png
../imgDataset/train/normal/normal_324.png => ../imgDataset/train/normal/normal_1799.png
../imgDataset/train/normal/normal_1675.png => ../imgDataset/train/normal/normal_1800.png
../imgDataset/train/normal/normal_1846.png => ../imgDataset/train/normal/normal_1801.png
../imgDataset/train/normal/normal_1852.png => ../imgDataset/train/normal/normal_1802.png
../imgDataset/train/normal/normal_495.png => ../imgDataset/train/normal/normal_1803.png
../imgDataset/train/normal/normal_481.png => ../imgDataset/train/normal/normal_1804.png
../imgDataset/train/normal/normal_668.png => ../imgDataset/train/normal/normal_1805.png
../imgDataset/train/normal/normal_1339.png => ../imgDataset/train/normal/normal_1806.png
../imgDataset/train/normal/normal_67.png => ../imgDataset/train/normal/normal_1807.png
../imgDataset/train/normal/n

../imgDataset/train/normal/normal_1662.png => ../imgDataset/train/normal/normal_1923.png
../imgDataset/train/normal/normal_1851.png => ../imgDataset/train/normal/normal_1924.png
../imgDataset/train/normal/normal_1689.png => ../imgDataset/train/normal/normal_1925.png
../imgDataset/train/normal/normal_1845.png => ../imgDataset/train/normal/normal_1926.png
../imgDataset/train/normal/normal_482.png => ../imgDataset/train/normal/normal_1927.png
../imgDataset/train/normal/normal_496.png => ../imgDataset/train/normal/normal_1928.png
../imgDataset/train/normal/normal_1879.png => ../imgDataset/train/normal/normal_1929.png
../imgDataset/train/normal/normal_1448.png => ../imgDataset/train/normal/normal_1930.png
../imgDataset/train/normal/normal_119.png => ../imgDataset/train/normal/normal_1931.png
../imgDataset/train/normal/normal_70.png => ../imgDataset/train/normal/normal_1932.png
../imgDataset/train/normal/normal_64.png => ../imgDataset/train/normal/normal_1933.png
../imgDataset/train/normal/n

In [41]:
changeName('../imgDataset/train/attack/','attack')

../imgDataset/train/attack/attack_556.png => ../imgDataset/train/attack/attack_0000.png
../imgDataset/train/attack/attack_1035.png => ../imgDataset/train/attack/attack_0001.png
../imgDataset/train/attack/attack_230.png => ../imgDataset/train/attack/attack_0002.png
../imgDataset/train/attack/attack_224.png => ../imgDataset/train/attack/attack_0003.png
../imgDataset/train/attack/attack_1021.png => ../imgDataset/train/attack/attack_0004.png
../imgDataset/train/attack/attack_542.png => ../imgDataset/train/attack/attack_0005.png
../imgDataset/train/attack/attack_1009.png => ../imgDataset/train/attack/attack_0006.png
../imgDataset/train/attack/attack_218.png => ../imgDataset/train/attack/attack_0007.png
../imgDataset/train/attack/attack_595.png => ../imgDataset/train/attack/attack_0008.png
../imgDataset/train/attack/attack_581.png => ../imgDataset/train/attack/attack_0009.png
../imgDataset/train/attack/attack_754.png => ../imgDataset/train/attack/attack_0010.png
../imgDataset/train/attack/at

../imgDataset/train/attack/attack_750.png => ../imgDataset/train/attack/attack_0314.png
../imgDataset/train/attack/attack_988.png => ../imgDataset/train/attack/attack_0315.png
../imgDataset/train/attack/attack_778.png => ../imgDataset/train/attack/attack_0316.png
../imgDataset/train/attack/attack_585.png => ../imgDataset/train/attack/attack_0317.png
../imgDataset/train/attack/attack_591.png => ../imgDataset/train/attack/attack_0318.png
../imgDataset/train/attack/attack_546.png => ../imgDataset/train/attack/attack_0319.png
../imgDataset/train/attack/attack_1025.png => ../imgDataset/train/attack/attack_0320.png
../imgDataset/train/attack/attack_220.png => ../imgDataset/train/attack/attack_0321.png
../imgDataset/train/attack/attack_234.png => ../imgDataset/train/attack/attack_0322.png
../imgDataset/train/attack/attack_1031.png => ../imgDataset/train/attack/attack_0323.png
../imgDataset/train/attack/attack_552.png => ../imgDataset/train/attack/attack_0324.png
../imgDataset/train/attack/att

../imgDataset/train/attack/attack_734.png => ../imgDataset/train/attack/attack_0416.png
../imgDataset/train/attack/attack_708.png => ../imgDataset/train/attack/attack_0417.png
../imgDataset/train/attack/attack_907.png => ../imgDataset/train/attack/attack_0418.png
../imgDataset/train/attack/attack_913.png => ../imgDataset/train/attack/attack_0419.png
../imgDataset/train/attack/attack_654.png => ../imgDataset/train/attack/attack_0420.png
../imgDataset/train/attack/attack_132.png => ../imgDataset/train/attack/attack_0421.png
../imgDataset/train/attack/attack_49.png => ../imgDataset/train/attack/attack_0422.png
../imgDataset/train/attack/attack_126.png => ../imgDataset/train/attack/attack_0423.png
../imgDataset/train/attack/attack_640.png => ../imgDataset/train/attack/attack_0424.png
../imgDataset/train/attack/attack_898.png => ../imgDataset/train/attack/attack_0425.png
../imgDataset/train/attack/attack_668.png => ../imgDataset/train/attack/attack_0426.png
../imgDataset/train/attack/attack

../imgDataset/train/attack/attack_1086.png => ../imgDataset/train/attack/attack_0553.png
../imgDataset/train/attack/attack_283.png => ../imgDataset/train/attack/attack_0554.png
../imgDataset/train/attack/attack_254.png => ../imgDataset/train/attack/attack_0555.png
../imgDataset/train/attack/attack_532.png => ../imgDataset/train/attack/attack_0556.png
../imgDataset/train/attack/attack_1051.png => ../imgDataset/train/attack/attack_0557.png
../imgDataset/train/attack/attack_1045.png => ../imgDataset/train/attack/attack_0558.png
../imgDataset/train/attack/attack_526.png => ../imgDataset/train/attack/attack_0559.png
../imgDataset/train/attack/attack_240.png => ../imgDataset/train/attack/attack_0560.png
../imgDataset/train/attack/attack_268.png => ../imgDataset/train/attack/attack_0561.png
../imgDataset/train/attack/attack_1079.png => ../imgDataset/train/attack/attack_0562.png
../imgDataset/train/attack/attack_518.png => ../imgDataset/train/attack/attack_0563.png
../imgDataset/train/attack/a

../imgDataset/train/attack/attack_844.png => ../imgDataset/train/attack/attack_0685.png
../imgDataset/train/attack/attack_81.png => ../imgDataset/train/attack/attack_0686.png
../imgDataset/train/attack/attack_878.png => ../imgDataset/train/attack/attack_0687.png
../imgDataset/train/attack/attack_56.png => ../imgDataset/train/attack/attack_0688.png
../imgDataset/train/attack/attack_893.png => ../imgDataset/train/attack/attack_0689.png
../imgDataset/train/attack/attack_887.png => ../imgDataset/train/attack/attack_0690.png
../imgDataset/train/attack/attack_139.png => ../imgDataset/train/attack/attack_0691.png
../imgDataset/train/attack/attack_42.png => ../imgDataset/train/attack/attack_0692.png
../imgDataset/train/attack/attack_111.png => ../imgDataset/train/attack/attack_0693.png
../imgDataset/train/attack/attack_677.png => ../imgDataset/train/attack/attack_0694.png
../imgDataset/train/attack/attack_663.png => ../imgDataset/train/attack/attack_0695.png
../imgDataset/train/attack/attack_1

../imgDataset/train/attack/attack_93.png => ../imgDataset/train/attack/attack_0830.png
../imgDataset/train/attack/attack_856.png => ../imgDataset/train/attack/attack_0831.png
../imgDataset/train/attack/attack_665.png => ../imgDataset/train/attack/attack_0832.png
../imgDataset/train/attack/attack_78.png => ../imgDataset/train/attack/attack_0833.png
../imgDataset/train/attack/attack_103.png => ../imgDataset/train/attack/attack_0834.png
../imgDataset/train/attack/attack_117.png => ../imgDataset/train/attack/attack_0835.png
../imgDataset/train/attack/attack_671.png => ../imgDataset/train/attack/attack_0836.png
../imgDataset/train/attack/attack_881.png => ../imgDataset/train/attack/attack_0837.png
../imgDataset/train/attack/attack_659.png => ../imgDataset/train/attack/attack_0838.png
../imgDataset/train/attack/attack_44.png => ../imgDataset/train/attack/attack_0839.png
../imgDataset/train/attack/attack_50.png => ../imgDataset/train/attack/attack_0840.png
../imgDataset/train/attack/attack_89

../imgDataset/train/attack/attack_588.png => ../imgDataset/train/attack/attack_0935.png
../imgDataset/train/attack/attack_775.png => ../imgDataset/train/attack/attack_0936.png
../imgDataset/train/attack/attack_1216.png => ../imgDataset/train/attack/attack_0937.png
../imgDataset/train/attack/attack_1202.png => ../imgDataset/train/attack/attack_0938.png
../imgDataset/train/attack/attack_761.png => ../imgDataset/train/attack/attack_0939.png
../imgDataset/train/attack/attack_991.png => ../imgDataset/train/attack/attack_0940.png
../imgDataset/train/attack/attack_749.png => ../imgDataset/train/attack/attack_0941.png
../imgDataset/train/attack/attack_985.png => ../imgDataset/train/attack/attack_0942.png
../imgDataset/train/attack/attack_952.png => ../imgDataset/train/attack/attack_0943.png
../imgDataset/train/attack/attack_946.png => ../imgDataset/train/attack/attack_0944.png
../imgDataset/train/attack/attack_167.png => ../imgDataset/train/attack/attack_0945.png
../imgDataset/train/attack/att

../imgDataset/train/attack/attack_968.png => ../imgDataset/train/attack/attack_1091.png
../imgDataset/train/attack/attack_32.png => ../imgDataset/train/attack/attack_1092.png
../imgDataset/train/attack/attack_149.png => ../imgDataset/train/attack/attack_1093.png
../imgDataset/train/attack/attack_26.png => ../imgDataset/train/attack/attack_1094.png
../imgDataset/train/attack/attack_613.png => ../imgDataset/train/attack/attack_1095.png
../imgDataset/train/attack/attack_175.png => ../imgDataset/train/attack/attack_1096.png
../imgDataset/train/attack/attack_161.png => ../imgDataset/train/attack/attack_1097.png
../imgDataset/train/attack/attack_607.png => ../imgDataset/train/attack/attack_1098.png
../imgDataset/train/attack/attack_834.png => ../imgDataset/train/attack/attack_1099.png
../imgDataset/train/attack/attack_820.png => ../imgDataset/train/attack/attack_1100.png
../imgDataset/train/attack/attack_808.png => ../imgDataset/train/attack/attack_1101.png
../imgDataset/train/attack/attack_

../imgDataset/train/attack/attack_18.png => ../imgDataset/train/attack/attack_1203.png
../imgDataset/train/attack/attack_30.png => ../imgDataset/train/attack/attack_1204.png
../imgDataset/train/attack/attack_639.png => ../imgDataset/train/attack/attack_1205.png
../imgDataset/train/attack/attack_24.png => ../imgDataset/train/attack/attack_1206.png
../imgDataset/train/attack/attack_942.png => ../imgDataset/train/attack/attack_1207.png
../imgDataset/train/attack/attack_956.png => ../imgDataset/train/attack/attack_1208.png
../imgDataset/train/attack/attack_765.png => ../imgDataset/train/attack/attack_1209.png
../imgDataset/train/attack/attack_1206.png => ../imgDataset/train/attack/attack_1210.png
../imgDataset/train/attack/attack_1212.png => ../imgDataset/train/attack/attack_1211.png
../imgDataset/train/attack/attack_771.png => ../imgDataset/train/attack/attack_1212.png
../imgDataset/train/attack/attack_759.png => ../imgDataset/train/attack/attack_1213.png
../imgDataset/train/attack/attack

In [23]:
print('Testset_attack :', len(os.listdir('../imgDataset/test/attack')))
print('Testset_normal :', len(os.listdir('../imgDataset/test/normal')))
print('Trainset_attack :', len(os.listdir('../imgDataset/train/attack')))
print('Trainset_normal :', len(os.listdir('../imgDataset/train/normal')))

Testset_attack : 299
Testset_normal : 494
Trainset_attack : 1225
Trainset_normal : 1948


In [24]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [25]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [26]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [27]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 이전 CNN 레이어에서 나온 3차원 배열은 1차원으로 뽑아줍니다
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [34]:
batch_size = 2

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다.
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        '../imgDataset/train',  # this is the target directory
        target_size=(128, 128),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        '../imgDataset/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        '../imgDataset/test',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

Found 3173 images belonging to 2 classes.
Found 793 images belonging to 2 classes.
Found 793 images belonging to 2 classes.


In [36]:
model.fit(train_generator, steps_per_epoch=batch_size, validation_data=validation_generator, epochs=1)
model.save_weights('conan_ver_02.h5')

ValueError: in user code:

    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/Caskroom/miniconda/base/envs/py_env/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_

# 파일명을 다 바꿔도 안된다.. 뭐가 문제일까!?